# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [30]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    states = []
    actions = []
    rewards = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    terminals = np.zeros(len(states))
    terminals[::100] = 1 #episode length 100, change if necessary
    print(states.shape)
    dataset = d3rlpy.dataset.MDPDataset(states.numpy(), 
                                        actions.numpy(), 
                                        rewards.numpy(), terminals)
    return dataset

We can build the dataset from there, just like this, and split into train and test sets.

In [47]:
dataset = get_dataset([3,5,7,9], path="collected_data/rl_det_small.txt")

[ 0.00000000e+00  7.95731469e+08 -8.17891077e-02 -1.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.09713430e-01 -2.63658359e-01  6.00000000e-01]
Read chunk # 4 out of 10000
[ 0.00000000e+00  7.95731469e+08  1.24610892e-01  2.40000469e-03
 -7.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.20016566e-01  3.79282423e-01 -6.00000000e-01]
Read chunk # 6 out of 10000
[ 0.00000000e+00  7.95731469e+08 -9.01891077e-02  1.08000047e-02
  3.99986580e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.17973088e-02 -2.40776052e-01  6.00000000e-01]
Read chunk # 8 out of 10000
[ 0.00000000e+00  7.95731469e+08  6.91108923e-02 -5.99999531e-03
 -6.00001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.42355434e-01  2.22081792e-01 -6.00000000e-01]
Read chunk # 10 out of 10000
torch.Size([111080, 6])


In [48]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -4.1227446,
 'std': 2.4676569,
 'min': -12.578855,
 'max': 0.0,
 'histogram': (array([ 26,   9,   7,   7,   8,   7,  10,  13,  27,  54,  56,  73, 109,
          84, 186, 148, 124,  83,  67,  13]),
  array([-12.578855 , -11.949912 , -11.320969 , -10.692026 , -10.063084 ,
          -9.434141 ,  -8.805199 ,  -8.176255 ,  -7.5473127,  -6.9183702,
          -6.2894273,  -5.6604843,  -5.031542 ,  -4.4025993,  -3.7736564,
          -3.1447136,  -2.515771 ,  -1.8868282,  -1.2578855,  -0.6289427,
           0.       ], dtype=float32))}

In [49]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)

## Setting up an Algorithm

In [50]:
from d3rlpy.dynamics import ProbabilisticEnsembleDynamics
from d3rlpy.metrics.scorer import dynamics_observation_prediction_error_scorer
from d3rlpy.metrics.scorer import dynamics_reward_prediction_error_scorer
from d3rlpy.metrics.scorer import dynamics_prediction_variance_scorer

dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=False)

# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=15, tensorboard_dir='runs',
             scorers={
                'observation_error': dynamics_observation_prediction_error_scorer,
                'reward_error': dynamics_reward_prediction_error_scorer,
                'variance': dynamics_prediction_variance_scorer,
             })


#from d3rlpy.algos import MOPO

# load trained dynamics model
#dynamics = ProbabilisticEnsembleDynamics.from_json('<path-to-params.json>/params.json')
#dynamics.load_model('<path-to-model>/model_xx.pt')

# give mopo as generator argument.
#mopo = MOPO(dynamics=dynamics)


2022-04-06 21:00.48 [debug    ] RoundIterator is selected.
2022-04-06 21:00.48 [info     ] Directory is created at d3rlpy_logs/ProbabilisticEnsembleDynamics_20220406210048
2022-04-06 21:00.48 [debug    ] Building models...
2022-04-06 21:00.48 [debug    ] Models have been built.
2022-04-06 21:00.48 [info     ] Parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220406210048/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'use_gpu': None, 'variance_type': 'max', 'algorithm': 'ProbabilisticEnsembleDynamics', 'observation_shape': 

Epoch 1/15:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:00.55 [info     ] ProbabilisticEnsembleDynamics_20220406210048: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00015752923787319036, 'time_algorithm_update': 0.006990694239362226, 'loss': -130.10344823763828, 'time_step': 0.007213200414913933, 'observation_error': 2.5002094379265388e-05, 'reward_error': 8.086704723235035e-05, 'variance': 2.9265567855343244e-08} step=878
2022-04-06 21:00.55 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220406210048/model_878.pt


Epoch 2/15:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:01.02 [info     ] ProbabilisticEnsembleDynamics_20220406210048: epoch=2 step=1756 epoch=2 metrics={'time_sample_batch': 0.0001431915374443037, 'time_algorithm_update': 0.006612565903022783, 'loss': -155.84424964713617, 'time_step': 0.006800605930337059, 'observation_error': 6.961562183296232e-05, 'reward_error': 5.49882593996615e-05, 'variance': 4.21097555442327e-07} step=1756
2022-04-06 21:01.02 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220406210048/model_1756.pt


...

Epoch 14/15:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:02.30 [info     ] ProbabilisticEnsembleDynamics_20220406210048: epoch=14 step=12292 epoch=14 metrics={'time_sample_batch': 0.00013669992633723996, 'time_algorithm_update': 0.00662463102362422, 'loss': -186.35415094154027, 'time_step': 0.0068031807423724125, 'observation_error': 1.2594006513682804e-05, 'reward_error': 3.0540305854605354e-06, 'variance': 2.454543805049739e-07} step=12292
2022-04-06 21:02.30 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220406210048/model_12292.pt


Epoch 15/15:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:02.37 [info     ] ProbabilisticEnsembleDynamics_20220406210048: epoch=15 step=13170 epoch=15 metrics={'time_sample_batch': 0.00014018985296436216, 'time_algorithm_update': 0.006717660432524453, 'loss': -188.058661486945, 'time_step': 0.0068993592859672255, 'observation_error': 1.0353658180396897e-05, 'reward_error': 2.9548844142438814e-06, 'variance': 3.397890133426663e-08} step=13170
2022-04-06 21:02.37 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20220406210048/model_13170.pt


[(1,
  {'time_sample_batch': 0.00015752923787319036,
   'time_algorithm_update': 0.006990694239362226,
   'loss': -130.10344823763828,
   'time_step': 0.007213200414913933,
   'observation_error': 2.5002094379265388e-05,
   'reward_error': 8.086704723235035e-05,
   'variance': 2.9265567855343244e-08}),
 (2,
  {'time_sample_batch': 0.0001431915374443037,
   'time_algorithm_update': 0.006612565903022783,
   'loss': -155.84424964713617,
   'time_step': 0.006800605930337059,
   'observation_error': 6.961562183296232e-05,
   'reward_error': 5.49882593996615e-05,
   'variance': 4.21097555442327e-07}),
 (3,
  {'time_sample_batch': 0.00015931710566909546,
   'time_algorithm_update': 0.007381611370008465,
   'loss': -163.18751751806306,
   'time_step': 0.007593790599588381,
   'observation_error': 1.946347884231508e-05,
   'reward_error': 3.987882613107416e-05,
   'variance': 5.701755339153832e-07}),
 (4,
  {'time_sample_batch': 0.00015286486892873987,
   'time_algorithm_update': 0.007048239740

In [12]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 77774), started 0:06:26 ago. (Use '!kill 77774' to kill it.)

In [51]:
from d3rlpy.algos import COMBO
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)



model = COMBO(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
        dynamics=dynamics,
          actor_learning_rate=3e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=5, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

7.825822524261004
2022-04-06 21:02.37 [debug    ] RoundIterator is selected.
2022-04-06 21:02.37 [info     ] Directory is created at d3rlpy_logs/COMBO_20220406210237
2022-04-06 21:02.37 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-06 21:02.37 [warning  ] Skip building models since they're already built.
2022-04-06 21:02.37 [info     ] Parameters are saved to d3rlpy_logs/COMBO_20220406210237/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 3e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'conservative_weight': 1.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'bet

Epoch 1/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-06 21:02.39 [debug    ] 250000 transitions are generated. fake_transitions=250000 real_transitions=87813
2022-04-06 21:02.47 [info     ] COMBO_20220406210237: epoch=1 step=343 epoch=1 metrics={'time_sample_batch': 0.00029832142087530465, 'time_algorithm_update': 0.022318651655325043, 'critic_loss': 8.361403107295578, 'actor_loss': -0.34950176004114025, 'temp_loss': 4.2451235399996925, 'temp': 0.9834807690656567, 'time_step': 0.022685573677974958, 'td_error': 0.7849345799815419, 'init_value': 1.2416236400604248, 'ave_value': 1.2467925996318105} step=343
2022-04-06 21:02.47 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20220406210237/model_343.pt


Epoch 2/5:   0%|          | 0/343 [00:00<?, ?it/s]

2022-04-06 21:02.56 [info     ] COMBO_20220406210237: epoch=2 step=686 epoch=2 metrics={'time_sample_batch': 0.0004691702283853692, 'time_algorithm_update': 0.022114493756530584, 'critic_loss': 10.902209612787987, 'actor_loss': -1.2672697422651265, 'temp_loss': 4.124696317984133, 'temp': 0.9538840698431255, 'time_step': 0.022652481457234818, 'td_error': 0.38186449550558743, 'init_value': 1.7204447984695435, 'ave_value': 1.7246237275531804} step=686
2022-04-06 21:02.56 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20220406210237/model_686.pt


...

Epoch 4/5:   0%|          | 0/686 [00:00<?, ?it/s]

2022-04-06 21:03.30 [debug    ] 250000 transitions are generated. fake_transitions=750000 real_transitions=87813
2022-04-06 21:03.32 [info     ] COMBO_20220406210237: epoch=4 step=2058 epoch=4 metrics={'time_sample_batch': 0.00047442760133882306, 'time_algorithm_update': 0.022584208594119234, 'critic_loss': 9.187148015631184, 'actor_loss': -4.4536585303854315, 'temp_loss': 3.6122775547010906, 'temp': 0.8502891756181467, 'time_step': 0.023136194871396433, 'td_error': 1.0253053341514484, 'init_value': 5.48727560043335, 'ave_value': 5.514245598049522} step=2058
2022-04-06 21:03.32 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20220406210237/model_2058.pt


Epoch 5/5:   0%|          | 0/686 [00:00<?, ?it/s]

2022-04-06 21:03.48 [info     ] COMBO_20220406210237: epoch=5 step=2744 epoch=5 metrics={'time_sample_batch': 0.0004726339012123753, 'time_algorithm_update': 0.021931777195054656, 'critic_loss': 9.289198818429218, 'actor_loss': -6.1472682966782815, 'temp_loss': 3.331260523017572, 'temp': 0.7984906840776216, 'time_step': 0.022479728081483536, 'td_error': 1.6410055861786605, 'init_value': 6.912630081176758, 'ave_value': 6.957811238080075} step=2744
2022-04-06 21:03.48 [info     ] Model parameters are saved to d3rlpy_logs/COMBO_20220406210237/model_2744.pt


[(1,
  {'time_sample_batch': 0.00029832142087530465,
   'time_algorithm_update': 0.022318651655325043,
   'critic_loss': 8.361403107295578,
   'actor_loss': -0.34950176004114025,
   'temp_loss': 4.2451235399996925,
   'temp': 0.9834807690656567,
   'time_step': 0.022685573677974958,
   'td_error': 0.7849345799815419,
   'init_value': 1.2416236400604248,
   'ave_value': 1.2467925996318105}),
 (2,
  {'time_sample_batch': 0.0004691702283853692,
   'time_algorithm_update': 0.022114493756530584,
   'critic_loss': 10.902209612787987,
   'actor_loss': -1.2672697422651265,
   'temp_loss': 4.124696317984133,
   'temp': 0.9538840698431255,
   'time_step': 0.022652481457234818,
   'td_error': 0.38186449550558743,
   'init_value': 1.7204447984695435,
   'ave_value': 1.7246237275531804}),
 (3,
  {'time_sample_batch': 0.0004773946267175257,
   'time_algorithm_update': 0.022580020629282247,
   'critic_loss': 9.356939674466414,
   'actor_loss': -2.4688512112934458,
   'temp_loss': 3.9773411139107306,


## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [52]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer

ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_det_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes, 
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08 -1.03891077e-02 -1.41999953e-02
 -2.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.78778459e-03 -1.34615461e-02  4.84073546e-02]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -7.24891077e-02 -1.35999953e-02
 -4.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.23311010e-02 -1.64283998e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08  7.01089229e-03 -4.19999531e-03
  7.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.21623335e-01 -2.86362315e-02 -8.00043364e-02]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01 -1.37999953e-02
  7.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.76352555e-01 -3.26280816e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-06 21:03.49 [debug    ] RoundIterator is selected.
2022-04-06 21:03.49 [info     ] Directory is created at d3rlpy_logs/FQE_2022040621034

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:03.50 [info     ] FQE_20220406210349: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00012873598546264927, 'time_algorithm_update': 0.001486791293550461, 'loss': 0.00026482629515046024, 'time_step': 0.0016694283431103127, 'init_value': -0.18876513838768005, 'ave_value': -0.18954894687402812, 'soft_opc': nan} step=878


2022-04-06 21:03.50 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210349/model_878.pt


Epoch 2/50:   0%|          | 0/878 [00:00<?, ?it/s]

...

2022-04-06 21:05.12 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210349/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:05.14 [info     ] FQE_20220406210349: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.00012210941531935152, 'time_algorithm_update': 0.0016487456671469304, 'loss': 0.012167671869002937, 'time_step': 0.0018187525060562447, 'init_value': -1.1216630935668945, 'ave_value': -1.120769039818994, 'soft_opc': nan} step=43900


2022-04-06 21:05.14 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210349/model_43900.pt


[(1,
  {'time_sample_batch': 0.00012873598546264927,
   'time_algorithm_update': 0.001486791293550461,
   'loss': 0.00026482629515046024,
   'time_step': 0.0016694283431103127,
   'init_value': -0.18876513838768005,
   'ave_value': -0.18954894687402812,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.0001276546838886374,
   'time_algorithm_update': 0.0015851612243130973,
   'loss': 0.0007146051946667641,
   'time_step': 0.0017693012884917726,
   'init_value': -0.2967626452445984,
   'ave_value': -0.2972891433198647,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00012643407852068578,
   'time_algorithm_update': 0.0016615939303248237,
   'loss': 0.0012968413317400534,
   'time_step': 0.0018410397553498218,
   'init_value': -0.4171195924282074,
   'ave_value': -0.4174483638273156,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00012486073313650075,
   'time_algorithm_update': 0.0015624658393425387,
   'loss': 0.00205733014416489,
   'time_step': 0.0017407282609874404,
  

In [55]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer

ope_dataset = get_dataset([2,4,6,8], path="collected_data/rl_stoch_small.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

fqe = FQE(algo=model, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes, 
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

[ 0.00000000e+00  7.95731469e+08  1.39310892e-01  1.82000047e-02
 -1.00013420e-04  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.50230117e-02  3.69851546e-01 -6.00000000e-01]
Read chunk # 3 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.15389108e-01  1.64000047e-02
 -8.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08831172e-01 -2.48178665e-01  6.00000000e-01]
Read chunk # 5 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.28589108e-01  1.20000047e-02
  1.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  7.75212759e-03 -3.52719043e-01  6.00000000e-01]
Read chunk # 7 out of 10000
[ 0.00000000e+00  7.95731469e+08 -1.03989108e-01  7.00000469e-03
 -8.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.65974295e-01 -2.19295880e-01  6.00000000e-01]
Read chunk # 9 out of 10000
torch.Size([111080, 6])
2022-04-06 21:07.38 [debug    ] RoundIterator is selected.
2022-04-06 21:07.38 [info     ] Directory is created at d3rlpy_logs/FQE_2022040621073

Epoch 1/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:07.40 [info     ] FQE_20220406210738: epoch=1 step=878 epoch=1 metrics={'time_sample_batch': 0.00011441077621216655, 'time_algorithm_update': 0.001423091714637426, 'loss': 0.0012160921468460657, 'time_step': 0.0015893791694032847, 'init_value': -0.48166346549987793, 'ave_value': -0.48177142936358197, 'soft_opc': nan} step=878


2022-04-06 21:07.40 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210738/model_878.pt


Epoch 2/50:   0%|          | 0/878 [00:00<?, ?it/s]

...

2022-04-06 21:08.54 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210738/model_43022.pt


Epoch 50/50:   0%|          | 0/878 [00:00<?, ?it/s]

2022-04-06 21:08.56 [info     ] FQE_20220406210738: epoch=50 step=43900 epoch=50 metrics={'time_sample_batch': 0.00011088554842868535, 'time_algorithm_update': 0.0013027755980611119, 'loss': 0.05379281517193776, 'time_step': 0.0014647383896256362, 'init_value': -2.247028350830078, 'ave_value': -2.247287266403565, 'soft_opc': nan} step=43900


2022-04-06 21:08.56 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220406210738/model_43900.pt


[(1,
  {'time_sample_batch': 0.00011441077621216655,
   'time_algorithm_update': 0.001423091714637426,
   'loss': 0.0012160921468460657,
   'time_step': 0.0015893791694032847,
   'init_value': -0.48166346549987793,
   'ave_value': -0.48177142936358197,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00011742630960729508,
   'time_algorithm_update': 0.0014501519518180968,
   'loss': 0.004580605509955626,
   'time_step': 0.001620275827638109,
   'init_value': -0.8132042288780212,
   'ave_value': -0.813346434212975,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00010611663375194088,
   'time_algorithm_update': 0.0013551312861518599,
   'loss': 0.010369343410954709,
   'time_step': 0.001508662532293715,
   'init_value': -1.1891367435455322,
   'ave_value': -1.1893279653407203,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00010223975214165272,
   'time_algorithm_update': 0.0012770095555820335,
   'loss': 0.016514954606140823,
   'time_step': 0.0014186293769262919,
   'in